# AlphaEarth / Google Satellite Embedding (V1) — Colombia (all years) extraction notebook

This notebook is focused on **processing / extracting** AlphaEarth Foundations **annual satellite embeddings** from Google Earth Engine, restricted to **Colombia**, in a way that avoids downloading massive GeoTIFF rasters.

**Key idea:** For image/pixel classification, you usually don't need the full 10 m embeddings raster for the whole country.
Instead, you can **sample the 64‑D embedding vector (bands `A00..A63`) at your labeled locations** (or at random points) and export those vectors as a compact table for model training.

Dataset used (Earth Engine):
- `ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")`  
- Annual coverage: **2017–2024** (one embedding field per calendar year)
- 10 m pixels, **64 bands** (`A00..A63`) per pixel (unit-length vectors)

Attribution (CC‑BY 4.0):  
> "The AlphaEarth Foundations Satellite Embedding dataset is produced by Google and Google DeepMind."

---

## What you will get out of this notebook

1. **A list of available years** in the collection.
2. A **Colombia region-of-interest** geometry.
3. A pipeline to:
   - Load your labeled points (CSV or GeoJSON)
   - For **each year**, extract the **64‑D embedding vector** at those points (`sampleRegions`)
   - Export results as **CSV** (Drive) or pull into **pandas** for smaller datasets

Optional:
- Create a **random sample** of embedding vectors across Colombia for unsupervised exploration.
- (Advanced) export **coarsened** embeddings (e.g., 250 m / 500 m) for broader mapping (still big, but more manageable).

---

## Requirements

- A Google Earth Engine account with access enabled.
- If running locally (not Colab): run `earthengine authenticate` once in your terminal.


In [5]:
# If you're running in a fresh environment, install dependencies.
# In Colab, you can run this cell once.
%pip -q install earthengine-api geemap pandas geopandas shapely pyarrow tqdm

import os
import ee
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import geemap

print("Libraries imported.")


Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [2]:
# --- Earth Engine authentication / initialization ---
# Colab: ee.Authenticate() will open a link + code flow
# Local Jupyter: usually ee.Initialize() works after `earthengine authenticate`

try:
    ee.Initialize()
    print("✅ Earth Engine initialized (existing credentials).")
except Exception as e:
    print("Earth Engine init failed:", e)
    print("\nRunning ee.Authenticate() ...")
    ee.Authenticate()
    ee.Initialize()
    print("✅ Earth Engine initialized (after authentication).")


Earth Engine init failed: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.

Running ee.Authenticate() ...



Successfully saved authorization token.
✅ Earth Engine initialized (after authentication).


## 1) Define Colombia ROI

We fetch Colombia boundaries from an Earth Engine administrative boundaries dataset and build a single geometry.


In [3]:
# Colombia boundary (GAUL simplified). Alternatives: USDOS/LSIB_SIMPLE/2017, etc.
countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

colombia = countries.filter(ee.Filter.eq("ADM0_NAME", "Colombia")).first()
colombia_geom = ee.Feature(colombia).geometry()

# Quick sanity check
print("Colombia feature:", colombia.getInfo()["properties"]["ADM0_NAME"])
print("Geometry type:", colombia_geom.getInfo()["type"])


Colombia feature: Colombia
Geometry type: GeometryCollection


In [4]:
# Visualize ROI
m = geemap.Map(center=[4.5, -74], zoom=5)
m.addLayer(colombia_geom, {}, "Colombia ROI")
m


Map(center=[4.5, -74], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

## 2) Load the Satellite Embedding V1 (annual) collection and list available years


In [6]:
emb_ic = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

# Restrict to Colombia footprint (this is only for counting/listing; sampling will also use this ROI)
emb_ic_co = emb_ic.filterBounds(colombia_geom)

# Get distinct years from system:time_start
years = (ee.List(emb_ic_co.aggregate_array("system:time_start"))
         .map(lambda t: ee.Date(t).get("year"))
         .distinct()
         .sort())

years_list = years.getInfo()
print("Available years (Colombia-intersecting tiles):", years_list)
print("Number of years:", len(years_list))

# Bands are A00..A63
first_img = ee.Image(emb_ic.first())
print("Band count:", len(first_img.bandNames().getInfo()))
print("First 5 bands:", first_img.bandNames().getInfo()[:5])


Available years (Colombia-intersecting tiles): [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
Number of years: 9
Band count: 64
First 5 bands: ['A00', 'A01', 'A02', 'A03', 'A04']


## 3) Helper functions

- Fetch an **annual embedding mosaic** for a year over Colombia.
- Extract embedding vectors at points (`sampleRegions`).
- Export tables to Drive.


In [7]:
def get_embedding_image_for_year(year: int, region: ee.Geometry) -> ee.Image:
    'Return a single ee.Image representing the annual embeddings for `year` over `region`.'
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, "year")
    # Tiles overlap the region; mosaic merges them into one image for sampling.
    img = (emb_ic
           .filterDate(start, end)
           .filterBounds(region)
           .mosaic())
    return img

def sample_embeddings_at_points(img: ee.Image, points_fc: ee.FeatureCollection,
                               properties=("label", "id"), scale=10) -> ee.FeatureCollection:
    'Attach the 64 embedding bands to each point feature.'
    return img.sampleRegions(
        collection=points_fc,
        properties=list(properties),
        scale=scale,
        geometries=True
    )

def export_featurecollection_to_drive(fc: ee.FeatureCollection, description: str,
                                      folder: str = "aef_embeddings",
                                      file_format: str = "CSV") -> ee.batch.Task:
    'Export a FeatureCollection as a table to Google Drive.'
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        description=description,
        folder=folder,
        fileFormat=file_format
    )
    task.start()
    return task

def print_active_tasks():
    'Print active EE tasks.'
    for t in ee.batch.Task.list():
        if t.status().get("state") in ("READY", "RUNNING"):
            print(t.status())


## 4) Load your labeled data (points)

**Recommended format (CSV):**
- `lon` (longitude)
- `lat` (latitude)
- `label` (your class label)
- optional: `id`

Place your file in the same folder as this notebook (or provide an absolute path).


In [ ]:
# ---- USER INPUT ----
LABELS_PATH = "labels_colombia.csv"   # <-- change this

# Example CSV schema:
# id,lon,lat,label
# 1,-74.0721,4.7110,urban
# 2,-75.5636,6.2518,forest

df = pd.read_csv(LABELS_PATH)
display(df.head())

required_cols = {"lon", "lat", "label"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in labels CSV: {missing}")

# Ensure an id exists
if "id" not in df.columns:
    df["id"] = np.arange(len(df))

# Convert to EE FeatureCollection
def row_to_feature(row):
    geom = ee.Geometry.Point([float(row["lon"]), float(row["lat"])])
    props = {"label": str(row["label"]), "id": int(row["id"])}
    return ee.Feature(geom, props)

features = [row_to_feature(r) for _, r in df.iterrows()]
labels_fc = ee.FeatureCollection(features)

print("Labeled points:", df.shape[0])


In [ ]:
# Visual check: points on the map
m2 = geemap.Map(center=[4.5, -74], zoom=5)
m2.addLayer(colombia_geom, {}, "Colombia ROI")
m2.addLayer(labels_fc, {"color": "red"}, "Labeled points")
m2


## 5) Extract embeddings for *all years* at your labeled points

Two options:

### Option A (small-ish data): pull into pandas (client-side)
Works well up to maybe tens of thousands of points *per year* (depends on quotas).

### Option B (recommended): export each year's samples to Google Drive
Scales much better.

> Note: Each output row will contain `A00..A63` plus `label`, `id`, and a geometry.


In [ ]:
# -------- Option A: client-side extraction into pandas (for small datasets) --------
# WARNING: This can be slow / memory-heavy if you have many points.

YEAR_TO_TEST = years_list[-1]  # latest available year
img = get_embedding_image_for_year(YEAR_TO_TEST, colombia_geom)
samples_fc = sample_embeddings_at_points(img, labels_fc, properties=("label", "id"), scale=10)

# Convert to pandas (geemap helper). You can also use samples_fc.getInfo() but it's less convenient.
samples_df = geemap.ee_to_pandas(samples_fc)
display(samples_df.head())

print("Rows:", len(samples_df))
print("Columns:", samples_df.columns[:10].tolist(), "...")


In [ ]:
# -------- Option B: export per year to Google Drive (recommended) --------
# This starts one Earth Engine export task per year.
# You can monitor task progress in the Earth Engine Tasks tab (Code Editor) or in Drive once complete.

DRIVE_FOLDER = "aef_embeddings_colombia"  # change if you want

tasks = []
for yr in years_list:
    img = get_embedding_image_for_year(yr, colombia_geom)
    samples_fc = sample_embeddings_at_points(img, labels_fc, properties=("label", "id"), scale=10)
    desc = f"aef_embeddings_colombia_labels_{yr}"
    task = export_featurecollection_to_drive(samples_fc, description=desc, folder=DRIVE_FOLDER, file_format="CSV")
    tasks.append(task)
    print("Started export:", desc)

print("\nActive tasks:")
print_active_tasks()


## 6) After download: turn the exported CSV into model-ready arrays

Each CSV has columns `A00..A63` plus your `label`/`id`. This cell shows how to load one export and make `X`/`y`.


In [ ]:
# ---- USER INPUT ----
DOWNLOADED_CSV = "aef_embeddings_colombia_labels_2024.csv"  # e.g., after you download from Drive

df_year = pd.read_csv(DOWNLOADED_CSV)

# Embedding columns
emb_cols = [f"A{str(i).zfill(2)}" for i in range(64)]
missing_emb = set(emb_cols) - set(df_year.columns)
if missing_emb:
    print("Missing embedding columns (check export):", sorted(list(missing_emb))[:10], "...")

X = df_year[emb_cols].to_numpy(dtype=np.float32)
y = df_year["label"].astype(str).to_numpy()

print("X shape:", X.shape)  # (n_samples, 64)
print("y shape:", y.shape)

# Optional: save to Parquet for faster re-loading
out_parquet = Path(DOWNLOADED_CSV).with_suffix(".parquet")
df_year.to_parquet(out_parquet, index=False)
print("Saved parquet:", out_parquet)


## 7) Optional: create a random sample of embeddings across Colombia (per year)

Useful for:
- Clustering / exploration
- Building a background dataset
- Semi-supervised methods

You choose how many points you want per year.


In [ ]:
# ---- USER INPUT ----
N_RANDOM_POINTS = 20000   # per year (adjust)
SEED = 42
YEAR_FOR_RANDOM = years_list[-1]

img = get_embedding_image_for_year(YEAR_FOR_RANDOM, colombia_geom)

# sample() creates random points and returns a FeatureCollection with band values
random_samples = img.sample(
    region=colombia_geom,
    numPixels=N_RANDOM_POINTS,
    seed=SEED,
    scale=10,
    geometries=True
)

# Export to Drive
desc = f"aef_embeddings_colombia_random_{YEAR_FOR_RANDOM}_{N_RANDOM_POINTS}"
task = export_featurecollection_to_drive(random_samples, description=desc, folder=DRIVE_FOLDER, file_format="CSV")
print("Started export:", desc)
print_active_tasks()


## 8) (Advanced) Coarsen embeddings before exporting rasters

Exporting a 10 m 64‑band raster for all of Colombia is huge.

If you *must* export raster embeddings, consider:
- **Coarsening** (e.g., 250 m / 500 m)
- **Quantizing** to int8 (lossy but often OK)

Below is an example of coarsening by averaging within blocks and re-normalizing to unit length.


In [ ]:
def renormalize_unit_length(img: ee.Image) -> ee.Image:
    'Renormalize a multi-band image so each pixel vector has L2 norm = 1.'
    l2 = img.pow(2).reduce(ee.Reducer.sum()).sqrt()
    return img.divide(l2)

def coarsen_embeddings(img: ee.Image, target_scale_m: int) -> ee.Image:
    '''
    Coarsen to `target_scale_m` by mean aggregation then renormalize.
    WARNING: Colombia spans multiple UTM zones, so for large exports you may want to reproject to EPSG:4326.
    '''
    reduced = img.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024)
    # Reproject to a scale (keeps native CRS unless you override)
    reduced = reduced.reproject(img.projection(), None, target_scale_m)
    return renormalize_unit_length(reduced)

YEAR_FOR_RASTER = years_list[-1]
TARGET_SCALE_M = 500  # try 250, 500, 1000, ...

img = get_embedding_image_for_year(YEAR_FOR_RASTER, colombia_geom)
img_coarse = coarsen_embeddings(img, TARGET_SCALE_M)

print("Coarsened image prepared.")
print("Native projection (example band):", img_coarse.select("A00").projection().getInfo())


### Raster export note

Earth Engine raster exports are subject to size limits. For large AOIs, export to Cloud Storage and/or tile the region.

This cell shows how you would *start* a Drive export for a **small AOI** (e.g., a department or bounding box).


In [ ]:
# Example: export a small AOI raster (replace with a smaller region than all Colombia!)
# Here we use a bounding box around Bogotá just as an example.

small_aoi = ee.Geometry.Rectangle([-74.4, 4.4, -73.8, 4.9])

task = ee.batch.Export.image.toDrive(
    image=img_coarse,
    description=f"aef_embeddings_raster_{YEAR_FOR_RASTER}_{TARGET_SCALE_M}m_smallAOI",
    folder=DRIVE_FOLDER,
    region=small_aoi,
    scale=TARGET_SCALE_M,
    maxPixels=1e13,
    fileFormat="GeoTIFF"
)
task.start()

print("Started raster export task (small AOI).")
print_active_tasks()


## 9) Next steps for your classification pipeline (outline)

Once you have the exported tables:
- Merge years if you want a **multi-year feature set** (e.g., concatenate embeddings from multiple years).
- Train a classifier (logistic regression, random forest, XGBoost, neural net) on `X` (64 features) → `label`.
- For mapping predictions, either:
  - apply the classifier inside Earth Engine (server-side), or
  - export a coarsened raster / tiles and run inference locally.

If you want, share:
- your label schema (classes),
- number of labeled points,
- and whether you need **pixel-level** maps or **region-level** classification,
and we can adapt the extraction/export strategy accordingly.
